# import

In [5]:
from pymatgen.io.cif import CifParser
from pymatgen.core.structure import Structure
from glob import glob
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer #sp
import pandas as pd
import os
import warnings
warnings.filterwarnings("ignore") 
#显示所有列
pd.set_option('display.max_columns', None)
#显示所有行
pd.set_option('display.max_rows',None)

cut_off = pd.read_csv('VESTA_cutoffs.csv')

# formula

In [9]:
#data
datas_mp = glob('*.cif')
result = pd.DataFrame()
for data_mp in datas_mp:
    name = os.path.basename(data_mp)
    name = name[:-4]
    result = result.append({'formula':name},ignore_index = True)
result.to_csv('formula.csv',index = False)

# bondlength

In [10]:
inputpath = '/Structure/'
outputpath = '/bondlengthcutoff/'
    
cif_files = glob(f'{inputpath}*.cif')

#CifParser
for cif_file in cif_files:
    name = os.path.basename(cif_file)
    name = name[:-4]
    parser = CifParser(cif_file)
    structure = parser.get_structures()[0]
    sym_finder = SpacegroupAnalyzer(structure) #sp
    structure = sym_finder.get_conventional_standard_structure()

    # calculate distance
    distances = structure.distance_matrix

    result = pd.DataFrame()

    # calculate bondlength
    for i in range(len(structure)):
        element_i = structure[i].specie.symbol
        if element_i != "O":
            for j in range(i+1, len(structure)):
                element_j = structure[j].specie.symbol
                if element_j == "O":
                    distance = distances[i][j]
                    result = result.append({'cation_site':f'{element_i}{i}', 'anion':element_j, 'distance':distance, 'cation':element_i},ignore_index= True)
    for i in range(len(structure)):
        element_i = structure[i].specie.symbol
        if element_i == "O":
            for j in range(i+1, len(structure)):
                element_j = structure[j].specie.symbol
                if element_j != "O":
                    distance = distances[i][j]
                    result = result.append({'cation_site':f'{element_j}{j}', 'anion':element_i, 'distance':distance, 'cation':element_j},ignore_index= True)
    re_li = pd.DataFrame()
    for row1 in result.values.tolist():
        cs, a, d, c = row1
        for row2 in cut_off.values.tolist():
            e1, e2, cf = row2
            if c == e1:
                if a == e2:
                    if d <= cf:
                        re_li = re_li.append({'cation_site':cs, 'anion':a, 'distance':d, 'cation':c},ignore_index= True)
    re_li.to_csv(f'{outputpath}{name}.csv',index = False)

spglib: No point group was found (line 405, /private/var/folders/3y/l3fp9ptn18j7k0mgn6pnkk1r0000gn/T/pip-install-xxn6rap4/spglib_8b203eccd15d4d73b8da004394912e84/src/pointgroup.c).
spglib: Attempt 0 tolerance = 1.000000e-02 failed(line 800, /private/var/folders/3y/l3fp9ptn18j7k0mgn6pnkk1r0000gn/T/pip-install-xxn6rap4/spglib_8b203eccd15d4d73b8da004394912e84/src/spacegroup.c).
spglib: No point group was found (line 405, /private/var/folders/3y/l3fp9ptn18j7k0mgn6pnkk1r0000gn/T/pip-install-xxn6rap4/spglib_8b203eccd15d4d73b8da004394912e84/src/pointgroup.c).
spglib: Attempt 1 tolerance = 9.500000e-03 failed(line 800, /private/var/folders/3y/l3fp9ptn18j7k0mgn6pnkk1r0000gn/T/pip-install-xxn6rap4/spglib_8b203eccd15d4d73b8da004394912e84/src/spacegroup.c).
spglib: No point group was found (line 405, /private/var/folders/3y/l3fp9ptn18j7k0mgn6pnkk1r0000gn/T/pip-install-xxn6rap4/spglib_8b203eccd15d4d73b8da004394912e84/src/pointgroup.c).
spglib: Attempt 0 tolerance = 1.000000e-02 failed(line 800, /pr